In [1]:
import ray
import re

# 初始化 Ray
ray.init()

@ray.remote
def find_longest_word(words):
    if not words:
        return ""
    return max(words, key=len)

def process_line(line):
    try:
        _, _, text = line.strip().split('\t', 2)  
    except ValueError:
        return []  
    
    words = re.findall(r'[A-Za-zА-Яа-я]+', text)
    return words


with open('wiki.txt', 'r', encoding='utf-8', errors='ignore') as file:
    longest_word_futures = [find_longest_word.remote(process_line(line)) for line in file]


longest_words = ray.get(longest_word_futures)
longest_word = max(longest_words, key=len)

print(f"The longest word in the document is: {longest_word}")


2024-05-08 23:27:18,965	INFO worker.py:1752 -- Started a local Ray instance.


The longest word in the document is: dreihundertvierundsechzigtausendachthundertneunzehn


In [2]:
import ray
import re

# ray.init()

@ray.remote
def average_word_length(words):
    if not words: 
        return (0, 0)
    lengths = [len(word) for word in words]
    return sum(lengths), len(lengths)

def process_line(line):
    try:
        _, _, text = line.strip().split('\t', 2) 
    except ValueError:
        return []
    
    words = re.findall(r'\b[а-яА-Яa-zA-Z]+\b', text)
    return words

with open('wiki.txt', 'r', encoding='utf-8', errors='ignore') as file:
    futures = [average_word_length.remote(process_line(line)) for line in file]

# 汇总计算结果
lengths_counts = ray.get(futures)
total_length = sum(length for length, count in lengths_counts)
total_count = sum(count for length, count in lengths_counts)
average_length = total_length / total_count if total_count else 0

print(f"The average word length in the document is: {average_length:.2f}")


The average word length in the document is: 6.25


In [3]:
import ray
import re
from collections import Counter

# ray.init()

@ray.remote
def most_common_latin_word(words):
    # 过滤出只包含拉丁字母的单词
    # Фильтрация слов, содержащих только латинские буквы
    latin_words = [word for word in words if word.isalpha() and word.encode().isascii()]
    # 统计频率
    # Статистическая частота
    counter = Counter(latin_words)
    return counter

def process_line(line):
    try:
        _, _, text = line.strip().split('\t', 2)
    except ValueError:
        return []
    
    words = re.findall(r'[A-Za-z]+', text)
    return words

with open('wiki.txt', 'r', encoding='utf-8', errors='ignore') as file:
    word_frequencies_futures = [most_common_latin_word.remote(process_line(line)) for line in file]

# count(слов)
all_counters = ray.get(word_frequencies_futures)
total_counter = Counter()
for counter in all_counters:
    total_counter.update(counter)

# max（частота слов）
most_common_word, frequency = total_counter.most_common(1)[0]
print(f"The most common Latin word is: {most_common_word} with {frequency} occurrences")


The most common Latin word is: formula with 11605 occurrences


In [4]:
import ray
import re
from collections import defaultdict, Counter

# ray.init()

@ray.remote
def analyze_words(words):
    capitalized_counts = Counter()  # 首字母大写的情况
    total_counts = Counter()        # 总出现次数

    for word in words:
        total_counts[word] += 1
        if word[0].isupper():
            capitalized_counts[word] += 1

    return capitalized_counts, total_counts

def process_line(line):
    try:
        _, _, text = line.strip().split('\t', 2)
    except ValueError:
        return []
    words = re.findall(r'\w+', text)
    return words

with open('wiki.txt', 'r', encoding='utf-8', errors='ignore') as file:
    futures = [analyze_words.remote(process_line(line)) for line in file]


capitalized_results = defaultdict(int)
total_results = defaultdict(int)

for future in ray.get(futures):
    capitalized_counts, total_counts = future
    for word, count in capitalized_counts.items():
        capitalized_results[word] += count
    for word, count in total_counts.items():
        total_results[word] += count

# Частота слов >10 && количество начальных заглавных букв >0,5
selected_words = [word for word in total_results if total_results[word] > 10 and 
                  (capitalized_results[word] / total_results[word] > 0.5)]

print(f"Words mostly capitalized and occur more than 10 times: {selected_words}")


Words mostly capitalized and occur more than 10 times: ['Литва', 'Респу', 'Северной', 'Европе', 'Центральной', 'Столица', 'Литвы', 'Вильнюс', 'Расположена', 'Балтийского', 'На', 'Латвией', 'Белоруссией', 'Польшей', 'Калининградской', 'России', 'Член', 'ООН', 'ЕС', 'НАТО', 'Входит', 'Шенгенскую', 'Поверхность', 'Поля', 'Высшая', 'Вильнюса', 'Более', 'Латвии', 'Белоруссии', 'Климат', 'Средняя', 'С', 'Название', 'Руси', 'Зарождение', 'X', 'XIII', 'Оно', 'Жемайтии', 'Уже', 'Великого', 'Литовского', 'Верхнего', 'Минском', 'В', 'Они', 'Пруссию', 'Ливонию', 'Галицко', 'Волынского', 'Из', 'Миндовга', 'Миндовг', 'Он', 'XIV', 'Чёрного', 'Тевтонским', 'Грюнвальдской', 'Королевства', 'Польского', 'Ягайло', 'Великое', 'Литовское', 'Королевство', 'Польское', 'Польши', 'Великим', 'Литовским', 'Витовт', 'Во', 'Казимир', 'Польше', 'Польшу', 'Пруссией', 'Чехию', 'Венгрию', 'Согласно', 'Люблинской', 'Литвой', 'Сейме', 'Однако', 'XVI', 'XVIII', 'Литве', 'Польско', 'ВКЛ', 'Россией', 'Австрией', 'Наполеона'

In [5]:
import ray
import re
from collections import Counter

# ray.init()

@ray.remote
def find_abbreviations(text):
    # re => "пр.", "др." 
    # (?:\s|^)：匹配一个空白字符或行的开头，确保缩写前面不是字母 
    # Убедитесь, что перед или после буквы нет букв.
    abbreviations = re.findall(r'(?:\s|^)[а-яА-Я]{2}\.(?![а-яА-Я])', text)
    return Counter(abbreviations)

def process_text(file_path):
    abbreviations_counter = Counter()
    with open(file_path, 'r', encoding='utf-8', errors='ignore') as file:
        
        futures = [find_abbreviations.remote(line.split('\t')[2]) for line in file if len(line.split('\t')) > 2]

    
    results = ray.get(futures)
    for result in results:
        abbreviations_counter.update(result)
    
    # слова >= 10
    frequent_abbreviations = {abbr: count for abbr, count in abbreviations_counter.items() if count >= 10}
    
    return frequent_abbreviations


abbreviations = process_text('wiki.txt')
print("Часто встречающиеся двухбуквенные аббревиатуры и количество их вхождений:", abbreviations)


Часто встречающиеся двухбуквенные аббревиатуры и количество их вхождений: {' СС.': 40, ' см.': 321, ' др.': 2393, ' юг.': 52, ' ст.': 401, '\xa0км.': 885, ' РФ.': 188, '\xa0гг.': 1102, ' мм.': 96, ' км.': 367, '\xa0кг.': 91, ' св.': 315, ' гг.': 1240, ' вв.': 211, '\xa0вв.': 450, ' кв.': 196, ' ЦК.': 25, ' им.': 912, ' Дж.': 293, ' ок.': 208, ' пр.': 222, ' ТВ.': 13, ' По.': 10, ' кн.': 42, ' их.': 158, '\xa0мм.': 249, ' ЕС.': 85, '\xa0см.': 163, ' Св.': 186, ' См.': 124, ' ср.': 79, ' оз.': 30, ' кг.': 59, ' га.': 190, ' ул.': 323, ' мн.': 41, ' ПК.': 26, ' ОС.': 25, ' пл.': 23, ' же.': 32, ' нм.': 40, ' фр.': 18, ' гр.': 21, ' эВ.': 11, ' ед.': 72, ' рт.': 23, ' СИ.': 35, ' об.': 34, ' шт.': 50, ' Гц.': 12, ' ум.': 26, ' мл.': 55, ' ПО.': 39, ' Си.': 21, ' ей.': 14, ' гл.': 26, ' тт.': 11, ' Вс.': 13, ' Вл.': 14, ' он.': 42, ' Ли.': 16, '\xa0мг.': 60, ' мг.': 26, ' Ср.': 12, ' Вт.': 12, ' Эл.': 10}


In [6]:
import ray
import re
from collections import Counter

# ray.init()

@ray.remote
def find_complex_abbreviations(text):
    # re => "т.п.", "н.э." 
    complex_abbreviations = re.findall(r'(?:\s|^)(?:[а-яА-Я]\. ){1}[а-яА-Я]\.(?![а-яА-Я])', text)
    return Counter(complex_abbreviations)

def process_text(file_path):
    abbreviations_counter = Counter()
    with open(file_path, 'r', encoding='utf-8', errors='ignore') as file:
        
        futures = [find_complex_abbreviations.remote(line.split('\t')[2]) for line in file if len(line.split('\t')) > 2]

    
    results = ray.get(futures)
    for result in results:
        abbreviations_counter.update(result)
    
    
    frequent_abbreviations = {abbr.replace('\xa0', ' '): count for abbr, count in abbreviations_counter.items() if count >= 10}
    
    return frequent_abbreviations


abbreviations = process_text('wiki.txt')
print("Часто встречающиеся двухбуквенные многословные аббревиатуры и количество их вхождений:", abbreviations)

ray.shutdown()

Часто встречающиеся двухбуквенные многословные аббревиатуры и количество их вхождений: {' с. ш.': 88, ' з. д.': 18, ' в. д.': 32, ' М. А.': 29, ' С. М.': 21, ' Н. И.': 23, ' Д. И.': 31, ' А. Е.': 12, ' М. С.': 16, ' В. В.': 46, ' А. Н.': 30, ' Н. М.': 14, ' т. д.': 92, ' т. п.': 45, ' И. В.': 24, ' А. И.': 48, ' А. Д.': 12, ' Л. Д.': 13, ' Ю. В.': 10, ' А. А.': 45, ' Н. Д.': 13, ' В. Н.': 21, ' М. В.': 40, ' И. И.': 18, ' В. А.': 44, ' С. В.': 21, ' В. И.': 46, ' А. П.': 25, ' А. Ф.': 19, ' А. В.': 38, ' К. Г.': 10, ' А. М.': 30, ' В. М.': 27, ' И. П.': 15, ' А. С.': 36, ' Г. А.': 12, ' В. Ф.': 15, ' Б. Н.': 19, ' Ф. М.': 10, ' В. П.': 16, ' в. н.': 18, ' н. э.': 881, ' г. н.': 22, ' ю. ш.': 39, ' т. е.': 38, ' Н. С.': 20, ' Н. А.': 27, ' т. н.': 18, ' Н. Н.': 29, ' а. е.': 61, ' Г. В.': 10, ' Л. И.': 20, ' Р. Х.': 14, ' В. Г.': 11, ' С. А.': 17, ' П. П.': 10, ' А. Г.': 19, ' л. с.': 45, ' П. А.': 18, ' П. И.': 12, ' И. А.': 15, ' В. С.': 11, ' А. К.': 10, ' П. Н.': 12, ' Д. А.': 10, '